In [1]:
from sdRDM import DataModel
from sdrdm_database import DBConnector, create_tables

In [2]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

In [3]:
# Fetch the sdRDM schema
lib = DataModel.from_markdown("./model.md")
lib.Root.meta_tree()

Root
├── id
├── value
└── nested
    └── 0
        ├── id
        └── another_value


ClassNode(/Root, class_name=None, constants={}, id=None, module=None, outer_type=None)

In [12]:
# Create tables for the sdRDM schema
create_tables(db_connector=db, model=lib.Root)

In [13]:
# We are now seeking a way to insert an instance of 
# the sdRDM schema into the database.
print(db.connection.table("Root"))
print(db.connection.table("nested"))

┏━━━━━━━━━┳━━━━━━━━━┓
┃ value   ┃ id      ┃
┡━━━━━━━━━╇━━━━━━━━━┩
│ float64 │ !string │
└─────────┴─────────┘



┏━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ another_value ┃ id      ┃ Root   ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ string        │ !string │ string │
└───────────────┴─────────┴────────┘



In [14]:
dataset1 = lib.Root(value=100.0)
dataset1.add_to_nested(another_value="something")
dataset1.add_to_nested(another_value="something else")

print(dataset1)

Root
├── id = root0
├── value = 100.0
└── nested
    ├── 0
    │   └── Nested
    │       ├── id = nested0
    │       └── another_value = something
    └── 1
        └── Nested
            ├── id = nested1
            └── another_value = something else



In [15]:
dataset2 = lib.Root(value=20.0)
dataset2.add_to_nested(another_value="hi there")
dataset2.add_to_nested(another_value="this is dataaset 2")

print(dataset2)

Root
├── id = root1
├── value = 20.0
└── nested
    ├── 0
    │   └── Nested
    │       ├── id = nested2
    │       └── another_value = hi there
    └── 1
        └── Nested
            ├── id = nested3
            └── another_value = this is dataaset 2



In [16]:
# Insert the new dataset into the database
db.insert(dataset1, dataset2, verbose=True)

Added dataset Root (41bf9d6a-1339-4f7b-a417-d7aec8d5e132)
Added dataset Root (8cffad8a-8757-4838-a814-1232c36b97e5)


In [17]:
# Check both tables
db.connection.table("Root")

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ value   ┃ id                                   ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ float64 │ !string                              │
├─────────┼──────────────────────────────────────┤
│   100.0 │ 41bf9d6a-1339-4f7b-a417-d7aec8d5e132 │
│    20.0 │ 8cffad8a-8757-4838-a814-1232c36b97e5 │
└─────────┴──────────────────────────────────────┘

In [18]:
db.connection.table("nested")

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ another_value      ┃ id                                   ┃ Root                                 ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string             │ !string                              │ string                               │
├────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│ this is dataaset 2 │ 08fe2c27-bf30-40a8-82ee-02b0440c6727 │ 8cffad8a-8757-4838-a814-1232c36b97e5 │
│ hi there           │ 461b8e72-e141-419a-b386-8f2d8639d357 │ 8cffad8a-8757-4838-a814-1232c36b97e5 │
│ something          │ bcf7e712-513f-4815-b40d-6d81cc589e17 │ 41bf9d6a-1339-4f7b-a417-d7aec8d5e132 │
│ something else     │ fbc6b7c4-1db7-45e4-8908-5a8befd0d0be │ 41bf9d6a-1339-4f7b-a417-d7aec8d5e132 │
└────────────────────┴──────────────────────────────────────┴──────────────────────────────────────┘